# 1.このSprintについて
### Sprintの目的
コンピュータビジョンの代表的タスクであるセグメンテーションを学ぶ  
### どのように学ぶか
公開されている実装を用いてKaggleコンペティションのデータでセグメンテーションを行います。  

# 2.セグメンテーション
KaggleのTGS Salt Identification Challengeのデータセットを使用し、セグメンテーションを行います。  
  
TGS Salt Identification Challenge | Kaggle  
  
セグメンテーション手法としてU-Net[1]を使います。  
  
[1]O.Ronneberger, P.Fischer, T.Brox. U-Net: Convolutional Networks for Biomedical Image Segmentation. Medical Image Computing and Computer-Assisted Intervention (MICCAI), Springer, LNCS, Vol.9351: 234–241, 2015  
  
https://arxiv.org/pdf/1505.04597.pdf

# 【問題1】学習・推定
以下のKeras実装を使用して学習・推定を行ってください。  
  
zhixuhao/unet: unet for image segmentation  
### 《GPU環境での学習》
大規模なデータセット、大きなモデルになるため、GPUを使用する必要があります。  
### 《新たなデータセットの適用》
公開されている実装で用意されたものとは異なるデータセットを入力するための準備が必要です。  

In [1]:
!pip install tensorflow==1.15

     |████████████████████████████████| 412.3MB 33kB/s 
     |████████████████████████████████| 3.8MB 40.1MB/s 
     |████████████████████████████████| 512kB 41.8MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=5f5c6a0d3d2b3c6303e3582788c09455a5ab26d023d9c7ff892d7ed7d017cab3
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: 

In [2]:
import tensorflow

In [3]:
tensorflow.__version__

'1.15.0'

In [4]:
!pip install kaggle

In [5]:
# 自分のマイドライブにマウントする

from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [6]:
cd /content/drive/"My Drive"

/content/drive/My Drive


In [7]:
# https://www.kaggle.com/　へアクセスし、ログイン後に右上のプロフィール画像をクリック、さらに"My Account"をクリックする
# このサイトの「API」項目にある"Create New API Token"をクリックすると、kaggle.jsonファイルが自動的にダウンロードされる
# ローカルで、kaggle.json（ダウンロードフォルダにあるはず）をエディターで開く。
# このセルの以下のコードにある token = {'username':'***','key':'***'} における「***」部分を、
# ダウンロードしたkaggle.jsonを参照して書き換え、このセルを実行する

import json

token = {"username":"hideakiishii","key":"2a74382de7f45456c1134d153e825a5c"}
with open('/content/drive/My Drive/kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [8]:
!chmod 600 '/content/drive/My Drive/kaggle/kaggle.json'

In [9]:
pwd

'/content/drive/My Drive'

In [10]:
ls -a kaggle

kaggle.json


In [11]:
mkdir　/root/.kaggle/

In [12]:
!cp -R '/content/drive/My Drive/kaggle/kaggle.json' /root/.kaggle/

In [13]:
!ls -a /root/.kaggle/

.  ..  kaggle.json


In [14]:
# kaggle APIコマンドが使えるかどうかを確認


!kaggle competitions list

ref                                            deadline             category            reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
tpu-getting-started                            2030-06-03 23:59:00  Getting Started      Kudos        122           False  
digit-recognizer                               2030-01-01 00:00:00  Getting Started  Knowledge       3018           False  
titanic                                        2030-01-01 00:00:00  Getting Started  Knowledge      23156            True  
house-prices-advanced-regression-techniques    2030-01-01 00:00:00  Getting Started  Knowledge       5064            True  
connectx                                       2030-01-01 00:00:00  Getting Started  Knowledge        668           False  
nlp-getting-started                            2030-01-01 00:00:00  Getting Started      Kudos       1496           False  
competit

In [15]:
# カレントディレクトリの変更

import os

os.chdir('/content/drive/My Drive/')

# カレントディレクトリの取得
print(os.getcwd())

/content/drive/My Drive


In [16]:
!kaggle competitions list -s TGS

ref                                deadline             category    reward  teamCount  userHasEntered  
---------------------------------  -------------------  --------  --------  ---------  --------------  
tgs-salt-identification-challenge  2018-10-19 23:59:00  Featured  $100,000       3229            True  


In [17]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay          69G   34G   32G  52% /
tmpfs            64M     0   64M   0% /dev
tmpfs           6.4G     0  6.4G   0% /sys/fs/cgroup
shm             5.8G     0  5.8G   0% /dev/shm
tmpfs           6.4G   16K  6.4G   1% /var/colab
/dev/sda1        75G   38G   38G  50% /opt/bin
tmpfs           6.4G     0  6.4G   0% /proc/acpi
tmpfs           6.4G     0  6.4G   0% /proc/scsi
tmpfs           6.4G     0  6.4G   0% /sys/firmware
drive            15G   12G  3.6G  77% /content/drive


In [18]:
# !kaggle competitions download -c tgs-salt-identification-challenge

In [19]:
# !unzip /content/drive/'My Drive'/train.zip  -d TGS_Salt_Identification

In [20]:
# !unzip /content/drive/'My Drive'/test.zip  -d TGS_Salt_Identification_test

In [21]:
# !unzip /content/drive/'My Drive'/flamingo.zip  -d TGS_Salt_Identification_flamingo

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
!git clone https://github.com/zhixuhao/unet.git

fatal: destination path 'unet' already exists and is not an empty directory.


In [24]:
cd /content/drive/'My Drive'/unet

/content/drive/My Drive/unet


In [25]:
ls -a

 data/               .git/    'main.py のコピー'   TGS_Salt_Identification/
 dataPrepare.ipynb   img/      model.py            trainUnet.ipynb
 data.py             LICENSE   __pycache__/        unet_membrane.hdf5
'data.py のコピー'   main.py   README.md


In [26]:
%cd /content/drive/'My Drive'/unet
!python /content/drive/'My Drive'/unet/main.py

/content/drive/My Drive/unet
Using TensorFlow backend.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

/content/drive/My Drive/unet/model.py:55: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = conv10)
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
2020-07-04 14:45:28.921588: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-07-04 14:45:28.972962: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-07-04 14:45:28.973779: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
20

# 【問題2】コードリーディング
論文[1]に目を通した上で、上記実装のコードリーディングを行ってください。

## ① エンコーダーの処理
「model.py」の以下のコードで、ダウンサンプリングを行っている

```python
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)
```

## ② デコーダーの処理
「model.py」の以下のコードでアップサンプリングを行っている  
concatenateではダウンサンプリングのデータがバイパスされている

```python
    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)
```